In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadrehan00/chest-xray-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'chest-xray-dataset' dataset.
Path to dataset files: /kaggle/input/chest-xray-dataset


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
import os
import sys

# --- 1. GPU & Memory Setup (CRITICAL) ---

# Verify GPU
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("❌ NO GPU DETECTED! Please go to Runtime > Change runtime type > T4 GPU")
    sys.exit() # Stop script if no GPU
else:
    print(f"✅ GPU Detected: {gpus[0].name}")

# Enable Mixed Precision (Faster training, less VRAM usage on T4)
mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed Precision Enabled")

# --- Configuration ---
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
EPOCHS = 20 # Increased epochs for fine-tuning
DATASET_PATH = "/kaggle/input/chest-xray-dataset/"

# Get class names and create mapping outside the graph for efficiency
def get_class_names(dataset_path):
    train_dir = os.path.join(dataset_path, 'train')
    # Use os.listdir and filter for directories to get class names
    return sorted([name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))])

class_names = get_class_names(DATASET_PATH)
num_classes = len(class_names)
class_to_idx = {name: i for i, name in enumerate(class_names)}
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_image(filepath):
    # Load the raw data from the file as a string
    img = tf.io.read_file(filepath)
    # Decode the image to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    img = tf.image.resize(img, IMG_SIZE)
    # EfficientNet expects input in [0, 255] range as float32
    img = tf.cast(img, tf.float32)
    return img

def get_label(file_path):
    # Convert path to string tensor
    parts = tf.strings.split(file_path, '/')
    # The label is the second to last part of the path (e.g., 'train/NORMAL/image.jpeg')
    label_str = parts[-2]
    # Use tf.py_function to execute Python code (dictionary lookup) within the TensorFlow graph
    return tf.py_function(lambda s: tf.constant(class_to_idx[s.numpy().decode('utf-8')], dtype=tf.int32),
                          [label_str], tf.int32)

def parse_image_and_label(filepath):
    img = preprocess_image(filepath)
    label = get_label(filepath)
    # Set static shapes for the tensors, which is good practice with tf.py_function
    img.set_shape((IMG_SIZE[0], IMG_SIZE[1], 3))
    label.set_shape([]) # Scalar label
    return img, label

def configure_for_performance(ds, shuffle=True):
    if shuffle:
        # Shuffle with a buffer, not the whole dataset to save RAM
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE) # Prefetch batches to keep GPU busy
    return ds

def build_and_train_model():
    print("Loading datasets...")

    train_dir = os.path.join(DATASET_PATH, 'train')
    val_dir = os.path.join(DATASET_PATH, 'val')

    # Get file paths using tf.data.Dataset.list_files for efficient listing
    # Adjusted glob pattern to search for '*.jpg' files instead of '*.jpeg'
    train_file_paths = tf.data.Dataset.list_files(os.path.join(train_dir, '*/*.jpg'))
    val_file_paths = tf.data.Dataset.list_files(os.path.join(val_dir, '*/*.jpg'))

    # Map preprocessing and label extraction functions in parallel
    train_ds = train_file_paths.map(parse_image_and_label, num_parallel_calls=AUTOTUNE)
    val_ds = val_file_paths.map(parse_image_and_label, num_parallel_calls=AUTOTUNE)

    # Configure for performance (batching, shuffling, prefetching)
    train_ds = configure_for_performance(train_ds, shuffle=True)
    val_ds = configure_for_performance(val_ds, shuffle=False)

    print(f"Classes found: {class_names}")

    # --- Data Augmentation ---
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])

    # --- Model Build (EfficientNetB0) ---
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )

    # Original compile with frozen base for initial training
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = data_augmentation(inputs)
    x = base_model(x, training=False) # Important: set training=False to keep BatchNormalization layers frozen
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\n--- Initial Training with Frozen Base Model ---")
    model.summary()

    # Optional: Add EarlyStopping to save time if model stops improving
    callback_frozen = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history_frozen = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10, # Fewer epochs for initial training
        callbacks=[callback_frozen]
    )

    # --- Fine-tuning: Unfreeze top layers of the base model ---
    print("\n--- Fine-tuning: Unfreezing top layers of EfficientNet ---")
    base_model.trainable = True

    # Unfreeze all layers except for the batch normalization layers
    for layer in base_model.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

    # Recompile model with a lower learning rate for fine-tuning
    # It's important to use a very small learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Very low learning rate
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    print("\nStarting fine-tuning...")
    callback_finetune = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history_finetune = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS, # Use remaining epochs or more
        callbacks=[callback_finetune]
    )

    # --- Save ---
    model.save('chest_xray_model_finetuned.keras')
    print("Model saved as chest_xray_model_finetuned.keras")

    with open('class_names.txt', 'w') as f:
        f.write('\n'.join(class_names))

if __name__ == "__main__":
    build_and_train_model()

✅ GPU Detected: /physical_device:GPU:0
✅ Mixed Precision Enabled
Loading datasets...
Classes found: ['normal', 'pneumonia', 'tuberculosis']

--- Initial Training with Frozen Base Model ---


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,040,579 (26.86 MB)

 Trainable params: 3,075 (12.01 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 249s 703ms/step - accuracy: 0.5066 - loss: 1.9594 - val_accuracy: 0.6906 - val_loss: 0.7235
Epoch 2/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 241s 722ms/step - accuracy: 0.6611 - loss: 0.7585 - val_accuracy: 0.6152 - val_loss: 0.9310
Epoch 3/10
 15/320 ━━━━━━━━━━━━━━━━━━━━ 3:10 623ms/step - accuracy: 0.6758 - loss: 0.7079

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('grand_slam_best_model.keras')
model.summary()